In [ ]:
!nvidia-smi

Sun Feb  6 20:40:51 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-SXM2...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   32C    P0    23W / 300W |      0MiB / 16160MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
# ! gdown --id 1i2x7osfloYkXDHMRn0WuVs2P3m751cj1
# ! pip -q uninstall -y kaggle
# ! pip -q install --upgrade pip
# ! pip -q install kaggle --upgrade
# ! mkdir ~/.kaggle
# ! cp kaggle.json ~/.kaggle/
# ! chmod 600 ~/.kaggle/kaggle.json
# ! kaggle competitions download jigsaw-toxic-severity-rating
# ! kaggle competitions download jigsaw-toxic-comment-classification-challenge
# ! kaggle datasets download aishikai/js-albert
# ! kaggle datasets download aishikai/js-roberta
# ! kaggle datasets download aishikai/fasttext-word-embeddings
# ! kaggle datasets download nkitgupta/jigsaw-regression-based-data
# ! kaggle datasets download aishikai/js-cleaned-validation-data
# ! kaggle datasets download rajkumarl/ruddit-jigsaw-dataset

 67% 5.00M/7.42M [00:00<00:00, 28.4MB/s]
100% 7.42M/7.42M [00:00<00:00, 38.3MB/s]


In [ ]:
# ! unzip /content/fasttext-word-embeddings.zip -d fasttext
# ! rm /content/fasttext-word-embeddings.zip
# ! unzip /content/jigsaw-regression-based-data.zip -d JS_Reg-Based-Data
# ! rm /content/jigsaw-regression-based-data.zip
# ! unzip /content/jigsaw-toxic-comment-classification-challenge.zip -d JTCC
# ! rm /content/jigsaw-toxic-comment-classification-challenge.zip
# ! unzip  /content/jigsaw-toxic-severity-rating.zip -d JTSR
# ! rm /content/jigsaw-toxic-severity-rating.zip
# ! unzip /content/js-albert.zip -d albert
# ! rm /content/js-albert.zip
# ! unzip /content/js-cleaned-validation-data.zip -d JTSR
# ! rm /content/js-cleaned-validation-data.zip
! unzip /content/ruddit-jigsaw-dataset.zip -d Ruddit
! rm /content/ruddit-jigsaw-dataset.zip

Archive:  /content/ruddit-jigsaw-dataset.zip
  inflating: Ruddit/Dataset/ReadMe.md  
  inflating: Ruddit/Dataset/Ruddit.csv  
  inflating: Ruddit/Dataset/Ruddit_individual_annotations.csv  
  inflating: Ruddit/Dataset/Thread_structure.txt  
  inflating: Ruddit/Dataset/create_dataset_variants.py  
  inflating: Ruddit/Dataset/identityterms_group.txt  
  inflating: Ruddit/Dataset/load_node_dictionary.py  
  inflating: Ruddit/Dataset/node_dictionary.npy  
  inflating: Ruddit/Dataset/post_with_issues.csv  
  inflating: Ruddit/Dataset/ruddit_with_text.csv  
  inflating: Ruddit/Dataset/sample_input_file.csv  
  inflating: Ruddit/LICENSE          
  inflating: Ruddit/Models/BERT.py   
  inflating: Ruddit/Models/BiLSTM.py  
  inflating: Ruddit/Models/HateBERT.py  
  inflating: Ruddit/Models/README.md  
  inflating: Ruddit/Models/create_splits.py  
  inflating: Ruddit/Models/info.md   
  inflating: Ruddit/README.md        
  inflating: Ruddit/requirements.txt  
  inflating: Ruddit/ruddit-comment

In [ ]:
# ! unzip /content/JTCC/sample_submission.csv.zip -d JTCC
# ! rm /content/JTCC/sample_submission.csv.zip
# ! unzip /content/JTCC/test.csv.zip -d JTCC
# ! rm /content/JTCC/test.csv.zip
# ! unzip /content/JTCC/test_labels.csv.zip -d JTCC
# ! rm /content/JTCC/test_labels.csv.zip
# ! unzip /content/JTCC/train.csv.zip -d JTCC
# ! rm /content/JTCC/train.csv.zip

In [ ]:
# ! pip -q install sentencepiece
# ! pip -q install transformers
# ! pip -q install wandb --upgrade
# ! pip -q install colorama
# ! pip install fasttext --upgrade

In [ ]:
import os
import gc
import copy
import time
import random
import pickle
import joblib
import string

# For data manipulation
import numpy as np
import pandas as pd

# Pytorch Imports
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torch.utils.data import Dataset, DataLoader

# Utils
from tqdm.notebook import tqdm
tqdm.pandas()
import wandb
wandb.login()
from collections import defaultdict

# Sklearn Imports
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import StratifiedKFold, KFold, GroupKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.linear_model import Ridge
from sklearn.metrics import accuracy_score, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer

# ML Imports
from xgboost import XGBRegressor

# NLP Imports
import re
import nltk
from nltk.corpus import stopwords
from nltk import WordNetLemmatizer
nltk.download('wordnet')
import itertools
from string import ascii_lowercase
import fasttext as ft
from bs4 import BeautifulSoup
from gensim.models import KeyedVectors, FastText

# For Transformer Models
from transformers import AutoTokenizer, AutoModel, AdamW, AutoConfig

# For colored terminal text
from colorama import Fore, Back, Style
b_ = Fore.BLUE
y_ = Fore.YELLOW
sr_ = Style.RESET_ALL

# Suppress warnings
import warnings
warnings.filterwarnings("ignore")

# For descriptive error messages
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [ ]:
def set_seed(seed=42):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    # When running on the CuDNN backend, two further options must be set
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # Set a fixed value for the hash seed
    os.environ['PYTHONHASHSEED'] = str(seed)
    
set_seed(42)

In [ ]:
df = pd.read_csv("/content/JTCC/train.csv")
df.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [ ]:
a = 1.0
b = 0.0
c = 0.0
d = 0.0
e = 0.0
f = 0.0
df["score"] = df["toxic"] * a + df["severe_toxic"] * b + df["identity_hate"] * c + df["insult"] * d + df["obscene"] * e + df["threat"] * f

In [ ]:
RE_PATTERNS = {
    ' american ':
        [
            'amerikan'
        ],

    ' adolf ':
        [
            'adolf'
        ],


    ' hitler ':
        [
            'hitler'
        ],

    ' fuck':
        [
            '(f)(u|[^a-z0-9 ])(c|[^a-z0-9 ])(k|[^a-z0-9 ])([^ ])*',
            '(f)([^a-z]*)(u)([^a-z]*)(c)([^a-z]*)(k)',
            ' f[!@#\$%\^\&\*]*u[!@#\$%\^&\*]*k', 'f u u c',
            '(f)(c|[^a-z ])(u|[^a-z ])(k)', r'f\*',
            'feck ', ' fux ', 'f\*\*', 
            'f\-ing', 'f\.u\.', 'f###', ' fu ', 'f@ck', 'f u c k', 'f uck', 'f ck','fuk', 'wtf','fucck','f cking'
        ],

    ' ass ':
        [
            '[^a-z]ass ', '[^a-z]azz ', 'arrse', ' arse ', '@\$\$'
                                                           '[^a-z]anus', ' a\*s\*s', '[^a-z]ass[^a-z ]',
            'a[@#\$%\^&\*][@#\$%\^&\*]', '[^a-z]anal ', 'a s s'
        ],

    ' asshole ':
        [
            ' a[s|z]*wipe', 'a[s|z]*[w]*h[o|0]+[l]*e', '@\$\$hole', 'ass hole'
        ],

    ' bitch ':
        [
            'b[w]*i[t]*ch', 'b!tch',
            'bi\+ch', 'b!\+ch', '(b)([^a-z]*)(i)([^a-z]*)(t)([^a-z]*)(c)([^a-z]*)(h)',
            'biatch', 'bi\*\*h', 'bytch', 'b i t c h','beetch'
        ],

    ' bastard ':
        [
            'ba[s|z]+t[e|a]+rd'
        ],

    ' transgender':
        [
            'transgender','trans gender'
        ],

    ' gay ':
        [
            'gay'
        ],

    ' cock ':
        [
            '[^a-z]cock', 'c0ck', '[^a-z]cok ', 'c0k', '[^a-z]cok[^aeiou]', ' cawk',
            '(c)([^a-z ])(o)([^a-z ]*)(c)([^a-z ]*)(k)', 'c o c k'
        ],

    ' dick ':
        [
            ' dick[^aeiou]', 'deek', 'd i c k','diick '
        ],

    ' suck ':
        [
            'sucker', '(s)([^a-z ]*)(u)([^a-z ]*)(c)([^a-z ]*)(k)', 'sucks', '5uck', 's u c k'
        ],

    ' cunt ':
        [
            'cunt', 'c u n t'
        ],

    ' bullshit ':
        [
            'bullsh\*t', 'bull\$hit','bs'
        ],

    ' homosexual':
        [
            'homo sexual','homosex'
        ],

    ' jerk ':
        [
            'jerk'
        ],

    ' idiot ':
        [
            'i[d]+io[t]+', '(i)([^a-z ]*)(d)([^a-z ]*)(i)([^a-z ]*)(o)([^a-z ]*)(t)', 'idiots', 'i d i o t'
        ],

    ' dumb ':
        [
            '(d)([^a-z ]*)(u)([^a-z ]*)(m)([^a-z ]*)(b)'
        ],

    ' shit ':
        [
            'shitty', '(s)([^a-z ]*)(h)([^a-z ]*)(i)([^a-z ]*)(t)', 'shite', '\$hit', 's h i t'
        ],

    ' shithole ':
        [
            'shythole','shit hole'
        ],

    ' retard ':
        [
            'returd', 'retad', 'retard', 'wiktard', 'wikitud'
        ],

    ' rape ':
        [
            ' raped'
        ],

    ' dumbass':
        [
            'dumb ass', 'dubass'
        ],

    ' asshead':
        [
            'butthead', 'ass head'
        ],

    ' sex ':
        [
            's3x', 'sexuality',
        ],


    ' nigger ':
        [
            'nigger', 'ni[g]+a', ' nigr ', 'negrito', 'niguh', 'n3gr', 'n i g g e r'
        ],

    ' shut the fuck up':
        [
            'stfu'
        ],

    ' pussy ':
        [
            'pussy[^c]', 'pusy', 'pussi[^l]', 'pusses'
        ],

    ' faggot ':
        [
            'faggot', ' fa[g]+[s]*[^a-z ]', 'fagot', 'f a g g o t', 'faggit',
            '(f)([^a-z ]*)(a)([^a-z ]*)([g]+)([^a-z ]*)(o)([^a-z ]*)(t)', 'fau[g]+ot', 'fae[g]+ot',
        ],

    ' motherfucker':
        [
            ' motha ', ' motha f', ' mother f', 'motherucker', 'mother fucker'
        ],

    ' whore ':
        [
            'wh\*\*\*', 'w h o r e'
        ],
}

In [ ]:
##Text Normalization

def clean_text(text, remove_repeat_text=True, remove_patterns_text=True, is_lower=True):
    
    template = re.compile(r'https?://\S+|www\.\S+') #Removes website links
    text = template.sub(r'', text)
    
    soup = BeautifulSoup(text, 'lxml') #Removes HTML tags
    only_text = soup.get_text()
    text = only_text
    
    emoji_pattern = re.compile("["
                               u"\U0001F600-\U0001F64F"  # emoticons
                               u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                               u"\U0001F680-\U0001F6FF"  # transport & map symbols
                               u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                               u"\U00002702-\U000027B0"
                               u"\U000024C2-\U0001F251"
                               "]+", flags=re.UNICODE)
    text = emoji_pattern.sub(r'', text)
    
    text = re.sub(r"[^a-zA-Z\d]", " ", text) #Remove special Charecters
    text = re.sub(' +', ' ', text) #Remove Extra Spaces
    text = text.strip()
    if is_lower:
        text=text.lower()

    if remove_patterns_text:
        for target, patterns in RE_PATTERNS.items():
          for pat in patterns:
            text=str(text).replace(pat, target)

    if remove_repeat_text:
        text = re.sub(r'(.)\1{2,}', r'\1', text) 
    
    text = str(text).replace("\n", " ")
    text = re.sub(r'[^\w\s]',' ',text)
    text = re.sub('[0-9]',"",text)
    text = re.sub(" +", " ", text)
    text = re.sub("([^\x00-\x7F])+"," ",text)
    return text 

In [ ]:
df["comment_text"] = df["comment_text"].progress_apply(clean_text)

  0%|          | 0/159571 [00:00<?, ?it/s]

In [ ]:
lemmatizer= WordNetLemmatizer()
def lemma(text, lemmatization=True):
  output=''
  if lemmatization:
    text=text.split(' ')
    for word in text:
      word1 = lemmatizer.lemmatize(word, pos = "n") #noun 
      word2 = lemmatizer.lemmatize(word1, pos = "v") #verb
      word3 = lemmatizer.lemmatize(word2, pos = "a") #adjective
      word4 = lemmatizer.lemmatize(word3, pos = "r") #adverb
      output=output + " " + word4
  else:
    output=text
  
  return str(output.strip())

In [ ]:
df["comment_text"] = df["comment_text"].progress_apply(lemma)

  0%|          | 0/159571 [00:00<?, ?it/s]

In [ ]:
df_sub["text"] = df_sub["text"].progress_apply(clean_text)
df_sub['text'] = df_sub["text"].progress_apply(clean)
df_sub["text"] = df_sub["text"].progress_apply(lemma)
df_sub["text"] = df_sub["text"].progress_apply(remove_stopwords)

In [ ]:
preds_less_toxic_["rb"] = preds_less_toxic_["roberta-base"] * 0.75 + preds_less_toxic_["roberta-base2"] * 0.25
preds_less_toxic_["rb"] = preds_less_toxic_["roberta-base"] * 0.75 + preds_less_toxic_["roberta-base2"] * 0.25

In [ ]:
##Stopwords Removal

stopword_list=[]
def iter_all_strings():
    for size in itertools.count(1):
        for s in itertools.product(ascii_lowercase, repeat=size):
            yield "".join(s)

dual_alpha_list=[]
for s in iter_all_strings():
    dual_alpha_list.append(s)
    if s == 'zz':
        break

dual_alpha_list.remove('i')
dual_alpha_list.remove('a')
dual_alpha_list.remove('am')
dual_alpha_list.remove('an')
dual_alpha_list.remove('as')
dual_alpha_list.remove('at')
dual_alpha_list.remove('be')
dual_alpha_list.remove('by')
dual_alpha_list.remove('do')
dual_alpha_list.remove('go')
dual_alpha_list.remove('he')
dual_alpha_list.remove('hi')
dual_alpha_list.remove('if')
dual_alpha_list.remove('is')
dual_alpha_list.remove('in')
dual_alpha_list.remove('me')
dual_alpha_list.remove('my')
dual_alpha_list.remove('no')
dual_alpha_list.remove('of')
dual_alpha_list.remove('on')
dual_alpha_list.remove('or')
dual_alpha_list.remove('ok')
dual_alpha_list.remove('so')
dual_alpha_list.remove('to')
dual_alpha_list.remove('up')
dual_alpha_list.remove('us')
dual_alpha_list.remove('we')


for letter in dual_alpha_list:
    stopword_list.append(letter)

In [ ]:
potential_stopwords=['editor', 'reference', 'thank', 'work','find', 'good', 'know', 'like', 'look', 'thing', 'want', 'time', 'list', 'section','wikipedia', 'doe', 'add','new', 'try', 'think', 'write','use', 'user', 'way', 'page']

#Adding above retrived words into the stopwords list.
for word in tqdm(potential_stopwords):
    stopword_list.append(word)

  0%|          | 0/25 [00:00<?, ?it/s]

In [ ]:
def remove_stopwords(text, remove_stop=True):
  output = ""
  if remove_stop:
    text=text.split(" ")
    for word in text:
      if word not in stopword_list:
        output=output + " " + word
  else :
    output=text

  return str(output.strip())

#Removing Stopwords from Train Data
df["comment_text"] = df["comment_text"].progress_apply(remove_stopwords)

  0%|          | 0/159571 [00:00<?, ?it/s]

### *FASTTEXT* Pretrained

In [ ]:
with open("fasttext.txt", "w") as fout:
    for sentence in df["comment_text"]:
        print(sentence, file = fout)

In [ ]:
%%time
model = ft.train_unsupervised(input="fasttext.txt",
                            model = "skipgram",
                            dim = 300,
                            ws = 5,
                            thread = 12,
                            verbose = 2,
                            epoch = 20)

CPU times: user 2h 57min 11s, sys: 10.2 s, total: 2h 57min 21s
Wall time: 22min 27s


In [ ]:
model.save_model("JS_fasttext.lib")

In [ ]:
fasttext_model = ft.load_model("/content/fasttext/crawl-300d-2M-subword.bin")

In [ ]:
fasttext_sent_embeddings = []
for _,row in tqdm(df.iterrows(), total = len(df)):
    fasttext_sent_embeddings.append(fasttext_model.get_sentence_vector(row["comment_text"]))

  0%|          | 0/159571 [00:00<?, ?it/s]

In [ ]:
fasttext_sent_embeddings = np.array(fasttext_sent_embeddings)

In [ ]:
y_true = df["score"].values

In [ ]:
# model = Ridge()
model = XGBRegressor(tree_method = "gpu_hist")
model.fit(fasttext_sent_embeddings, y_true)

[21:39:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


XGBRegressor(tree_method='gpu_hist')

In [ ]:
val_df = pd.read_csv("/content/JTSR/Validation_data_clean_no_duplicates.csv")
val_df.head()

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"


In [ ]:
val_df["truth"] = 1

In [ ]:
if VAL:
    df_val["more_toxic"] = df_val["more_toxic"].progress_apply(clean_text)
    df_val['more_toxic'] = df_val["more_toxic"].progress_apply(clean)
    df_val["more_toxic"] = df_val["more_toxic"].progress_apply(lemma)
    df_val["more_toxic"] = df_val["more_toxic"].progress_apply(remove_stopwords)

In [ ]:
val_df["less_toxic"] = val_df["less_toxic"].progress_apply(clean_text)
val_df["less_toxic"] = val_df["less_toxic"].progress_apply(lemma)
val_df["less_toxic"] = val_df["less_toxic"].progress_apply(remove_stopwords)
val_df["more_toxic"] = val_df["more_toxic"].progress_apply(clean_text)
val_df["more_toxic"] = val_df["more_toxic"].progress_apply(lemma)
val_df["more_toxic"] = val_df["more_toxic"].progress_apply(remove_stopwords)

  0%|          | 0/24808 [00:00<?, ?it/s]

  0%|          | 0/24808 [00:00<?, ?it/s]

  0%|          | 0/24808 [00:00<?, ?it/s]

  0%|          | 0/24808 [00:00<?, ?it/s]

  0%|          | 0/24808 [00:00<?, ?it/s]

  0%|          | 0/24808 [00:00<?, ?it/s]

In [ ]:
fasttext_sent_embeddings_more = []
fasttext_sent_embeddings_less = []
for _,row in tqdm(val_df.iterrows(), total = len(val_df)):
    fasttext_sent_embeddings_more.append(fasttext_model.get_sentence_vector(row["more_toxic"]))
    fasttext_sent_embeddings_less.append(fasttext_model.get_sentence_vector(row["less_toxic"]))
fasttext_sent_embeddings_more = np.array(fasttext_sent_embeddings_more)
fasttext_sent_embeddings_less = np.array(fasttext_sent_embeddings_less)

  0%|          | 0/24808 [00:00<?, ?it/s]

In [ ]:
pred_more_toxic = model.predict(fasttext_sent_embeddings_more)
pred_less_toxic = model.predict(fasttext_sent_embeddings_less)
predictions = pred_more_toxic > pred_less_toxic
val_truths = val_df.truth.values
print(f"Accuracy: {accuracy_score(val_truths, predictions)}")

Accuracy: 0.7173895517574976


In [ ]:
fasttext_preds = pd.DataFrame()
fasttext_preds["more"] = pred_more_toxic
fasttext_preds["less"] = pred_less_toxic
fasttext_preds.to_csv("fasttext_preds.csv", index=None)

In [ ]:
test_df = pd.read_csv("/content/JTSR/comments_to_score.csv")
test_df.head()

,comment_id,text
0,114890,"""\n \n\nGjalexei, you asked about whether ther..."
1,732895,"Looks like be have an abuser , can you please ..."
2,1139051,I confess to having complete (and apparently b...
3,1434512,"""\n\nFreud's ideas are certainly much discusse..."
4,2084821,It is not just you. This is a laundry list of ...


In [ ]:
test_df["text"] = test_df["text"].progress_apply(clean_text)
test_df["text"] = test_df["text"].progress_apply(lemma)
test_df["text"] = test_df["text"].progress_apply(remove_stopwords)

  0%|          | 0/7537 [00:00<?, ?it/s]

  0%|          | 0/7537 [00:00<?, ?it/s]

  0%|          | 0/7537 [00:00<?, ?it/s]

In [ ]:
fasttext_sent_embeddings_test = []
for _,row in tqdm(test_df.iterrows(), total = len(test_df)):
    fasttext_sent_embeddings_test.append(fasttext_model.get_sentence_vector(row["text"]))
fasttext_sent_embeddings_test = np.array(fasttext_sent_embeddings_test)

  0%|          | 0/7537 [00:00<?, ?it/s]

In [ ]:
test_preds = model.predict(fasttext_sent_embeddings_test)

In [ ]:
test_df["score"] = test_preds
# test_df['score'] = test_df['score'].rank(method='first')
test_df[['comment_id','score']].to_csv("roberta_.csv", index=None)

In [ ]:
test_df.head()

,comment_id,text,score
0,114890,gjalexei you ask about whether there be an ant...,0.017395
1,732895,be have an abuser can you please into this,0.093389
2,1139051,i confess to have complete and apparently blis...,-0.002968
3,1434512,freud idea be certainly much discuss today and...,0.049988
4,2084821,be not just you this be a laundry of stupid al...,0.134944


In [ ]:
# def train():     
#     with wandb.init() as _:
      
#         model = XGBRegressor(
            
#             tree_method = "gpu_hist",
#             n_estimators=wandb.config['n_estimators'],  
#             max_depth=wandb.config['max_depth']   ,
#             learning_rate = wandb.config['learning_rate'],
#             colsample_bytree = wandb.config['colsample_bytree'],
#             gamma =  wandb.config['gamma'],
#             n_jobs = -1
#             # [Optional] add additional model parameters here
            
#             )

#         # ✍️ Log your Models parameter config to W&B
#         wandb.config['model_type'] = 'XGBRegresor'
#         wandb.config.update(model.get_params())

#         model.fit(fasttext_sent_embeddings, y_true)

#         pred_more_toxic = model.predict(fasttext_sent_embeddings_more)
#         pred_less_toxic = model.predict(fasttext_sent_embeddings_less)
#         predictions = pred_more_toxic > pred_less_toxic

#         # Log validation summary metrics to W&B
#         wandb.summary["validation/accuracy"] = accuracy_score(val_truths, predictions)
#         wandb.summary["validation/f1_score"] = f1_score(val_truths, predictions, average="weighted")

#         # Make test set predictions and save as csv  
#         y_pred_test = model.predict(fasttext_sent_embeddings_test)

#         test_df["score"] = test_preds
#         test_df['score'] = test_df['score'].rank(method='first')
#         test_df[['comment_id','score']].to_csv("submission.csv", index=None)

#         wandb.log_artifact('submission.csv', name=f'{wandb.run.id}_submission', type='submission')

In [ ]:
# PROJECT = "JS"

In [ ]:
# sweep_config = {
#   "name" : "Toxicity_sweep",
#   "method" : "bayes",
#   'metric': {
#          'name': 'validation/accuracy',
#          'goal': 'maximize'  
#        },
#   "parameters" : {
#     "n_estimators" :{
#       "min": 50,
#       "max": 500
#     },
#     "max_depth" :{
#       "min": 3,
#       "max": 20
#     },
#     "learning_rate":{
#         "min": 1e-4,
#         "max":0.1
#     },
#     "colsample_bytree":{
#         "min":0.2,
#         "max":0.8
#     },
#     "gamma":{
#         "max": 5,
#         "min": 0
#     }

#     # [Optional] add additional parameters here

#   }
# }

# sweep_id = wandb.sweep(sweep_config, project=PROJECT)

In [ ]:
# N_RUNS = 50 # number of runs to execute
# wandb.agent(sweep_id, project=PROJECT, function=train, count=N_RUNS)

In [ ]:
del model, fasttext_model
gc.collect()

9552

### *TF-IDF* Jigsaw

In [ ]:
%%time
vec = TfidfVectorizer(min_df= 3, max_df=0.5, analyzer = 'char_wb', ngram_range = (3,5), max_features = 46000)
vec.fit(df['comment_text'])

CPU times: user 1min 14s, sys: 389 ms, total: 1min 15s
Wall time: 1min 14s


In [ ]:
joblib.dump(vec, "JS_tfidf.pkl")

['JS_tfidf.pkl']

In [ ]:
train_tfidf = vec.transform(df["comment_text"])

In [ ]:
%%time
model = Ridge(alpha=0.5)
model.fit(train_tfidf, df['score'])

CPU times: user 45.1 s, sys: 2min 5s, total: 2min 51s
Wall time: 26.3 s


In [ ]:
pickle.dump(model, open("JS_Ridge.pkl", 'wb'))

In [ ]:
%%time
more_toxic_tfidf = vec.transform(val_df["more_toxic"])
less_toxic_tfidf = vec.transform(val_df["less_toxic"])

CPU times: user 23.7 s, sys: 177 ms, total: 23.9 s
Wall time: 23.7 s


In [ ]:
more_toxic_preds = model.predict(more_toxic_tfidf)
less_toxic_preds = model.predict(less_toxic_tfidf)

In [ ]:
predictions = more_toxic_preds > less_toxic_preds
val_truths = val_df.truth.values
print(f"Accuracy: {accuracy_score(val_truths, predictions)}")

Accuracy: 0.7487100935182199


In [ ]:
tfidf_preds = pd.DataFrame()
tfidf_preds["more"] = more_toxic_preds
tfidf_preds["less"] = less_toxic_preds
tfidf_preds.to_csv("tfidf_preds.csv", index=None)

### TF-IDF Ruddit

In [ ]:
df_ruddit = pd.read_csv("/content/Ruddit/Dataset/ruddit_with_text.csv")
print(df_ruddit.shape)
df_ruddit.sample(5)

(5838, 5)


,post_id,comment_id,txt,url,offensiveness_score
834,9op9bu,e7wumsc,"I'm, like, a really manly guy. Everyone says s...",https://www.reddit.com/r/changemyview/comments...,-0.021
1288,amrwus,eg1lari,That is a pretty large generalization since no...,https://www.reddit.com/r/changemyview/comments...,-0.333
5342,bdlby1,ekz5xhg,"Compared to the previous century, I think we'r...",https://www.bbc.co.uk/news/live/world-europe-4...,-0.375
2131,64qx7m,dg4d6e8,This was the late 90's so maybe the view had s...,https://www.reddit.com/r/AskReddit/comments/64...,-0.396
285,7ej2uh,dq5bvdz,"okay counter argument, when is a city forced?\...",https://www.reddit.com/r/changemyview/comments...,-0.021


In [ ]:
df_ruddit = df_ruddit.loc[df_ruddit.txt != "[deleted]" ]
print(df_ruddit.shape)

(5722, 5)


In [ ]:
df_ruddit["txt"] = df_ruddit["txt"].progress_apply(clean_text)
df_ruddit["txt"] = df_ruddit["txt"].progress_apply(lemma)
df_ruddit["txt"] = df_ruddit["txt"].progress_apply(remove_stopwords)

  0%|          | 0/5722 [00:00<?, ?it/s]

  0%|          | 0/5722 [00:00<?, ?it/s]

  0%|          | 0/5722 [00:00<?, ?it/s]

In [ ]:
vec2 = TfidfVectorizer(analyzer='char_wb', max_df=0.7, min_df=3, ngram_range=(3, 4) )

In [ ]:
vec2.fit(df_ruddit["txt"])

TfidfVectorizer(analyzer='char_wb', max_df=0.7, min_df=3, ngram_range=(3, 4))

In [ ]:
joblib.dump(vec2, "JS_tfidf_ruddit.pkl")

['JS_tfidf_ruddit.pkl']

In [ ]:
ruddit_tfidf = vec2.transform(df_ruddit["txt"])

In [ ]:
%%time
model2 = Ridge(alpha=0.1)
model2.fit(ruddit_tfidf, df_ruddit['offensiveness_score'])

CPU times: user 542 ms, sys: 1.79 s, total: 2.33 s
Wall time: 310 ms


In [ ]:
pickle.dump(model2, open("JS_Ridge_ruddit.pkl", 'wb'))

In [ ]:
more_toxic_tfidf2 = vec2.transform(val_df["more_toxic"])
less_toxic_tfidf2 = vec2.transform(val_df["less_toxic"])

In [ ]:
more_toxic_preds2 = model2.predict(more_toxic_tfidf2)
less_toxic_preds2 = model2.predict(less_toxic_tfidf2)

In [ ]:
predictions = more_toxic_preds2 > less_toxic_preds2
val_truths = val_df.truth.values
print(f"Accuracy: {accuracy_score(val_truths, predictions)}")

Accuracy: 0.6806675266043212


### *DL*

In [ ]:
CONFIG = dict(
    seed = 42,
    config_name = '/content/albert/albert-large-v2/albert-large-v2/config.json',
    model_name = '/content/albert/albert-large-v2/albert-large-v2',
    test_batch_size = 32,
    max_length = 128,
    num_classes = 1,
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu"),
    debug= False,
    size = 1024
)

CONFIG["tokenizer"] = AutoTokenizer.from_pretrained(CONFIG['model_name'])

In [ ]:
class JigsawDataset(Dataset):
    def __init__(self, df, tokenizer, max_length, col):
        self.df = df
        self.max_len = max_length
        self.tokenizer = tokenizer
        self.text = df[col].values
        
    def __len__(self):
        return len(self.df)
    
    def __getitem__(self, index):
        text = self.text[index]
        inputs = self.tokenizer.encode_plus(
                        text,
                        truncation=True,
                        add_special_tokens=True,
                        max_length=self.max_len,
                        padding='max_length'
                    )
        
        ids = inputs['input_ids']
        mask = inputs['attention_mask']        
        
        return {
            'ids': torch.tensor(ids, dtype=torch.long),
            'mask': torch.tensor(mask, dtype=torch.long)
        }

In [ ]:
# class AttentionHead(nn.Module):
#     def __init__(self, in_features, hidden_dim):
#         super().__init__()
#         self.in_features = in_features
#         self.middle_features = hidden_dim
#         self.W = nn.Linear(in_features, hidden_dim)
#         self.V = nn.Linear(hidden_dim, 1)
#         self.out_features = hidden_dim

#     def forward(self, features):
#         att = torch.tanh(self.W(features))
#         score = self.V(att)
#         attention_weights = torch.softmax(score, dim=1)
#         context_vector = attention_weights * features
#         context_vector = torch.sum(context_vector, dim=1)

#         return context_vector

# class JigsawModel(nn.Module):
#     def __init__(self, model_name, config_path=None, pretrained=False):
#         super().__init__()
#         self.config = AutoConfig.from_pretrained(model_name, output_hidden_states=True)
#         self.model = AutoModel.from_config(config=self.config)
#         self.head = AttentionHead(self.config.hidden_size,self.config.hidden_size)
#         self.dropout = nn.Dropout(0.4)
#         self.linear = nn.Linear(self.config.hidden_size,1)

#     def forward(self, ids, mask):
#         x = self.model(input_ids=ids,attention_mask=mask)[0]
#         x = self.head(x)
#         x = self.dropout(x)
#         x = self.linear(x)
#         return x

In [ ]:
class JigsawModel(nn.Module):
    def __init__(self, config_name):
        super(JigsawModel, self).__init__()
        config = AutoConfig.from_pretrained(config_name)
        self.model = AutoModel.from_config(config=config)
        self.drop = nn.Dropout(p=0.2)
        self.fc = nn.Linear(CONFIG['size'], CONFIG['num_classes'])
        
    def forward(self, ids, mask):        
        out = self.model(input_ids=ids,attention_mask=mask,
                         output_hidden_states=False)
        out = self.drop(out[1])
        outputs = self.fc(out)
        return outputs

    def _get_features(self, ids, mask):
        x = self.model(input_ids=ids,attention_mask=mask)[1]
        return x

In [ ]:
val_df = pd.read_csv("/content/JTSR/Validation_data_clean_no_duplicates.csv")
if CONFIG["debug"]:
    val_df = val_df[:100]
val_df.head()

,worker,less_toxic,more_toxic
0,313,This article sucks \n\nwoo woo wooooooo,WHAT!!!!!!!!?!?!!?!?!!?!?!?!?!!!!!!!!!!!!!!!!!...
1,188,"""And yes, people should recognize that but the...",Daphne Guinness \n\nTop of the mornin' my fav...
2,82,"Western Media?\n\nYup, because every crime in...","""Atom you don't believe actual photos of mastu..."
3,347,And you removed it! You numbskull! I don't car...,You seem to have sand in your vagina.\n\nMight...
4,539,smelly vagina \n\nBluerasberry why don't you ...,"hey \n\nway to support nazis, you racist"


In [ ]:
MODEL_PATHS = [f"/content/albert/JS_Albert-v2L_RL/models/albert-large-v2_{fold}_model.bin" for fold in range(5)]

In [ ]:
@torch.no_grad()
def valid_fn(model, dataloader, device):
    model.eval()
    
    dataset_size = 0
    running_loss = 0.0
    
    PREDS = []
    
    bar = tqdm(enumerate(dataloader), total=len(dataloader))
    for step, data in bar:
        ids = data['ids'].to(device, dtype = torch.long)
        mask = data['mask'].to(device, dtype = torch.long)
        
        outputs = model(ids, mask)
        PREDS.append(outputs.sigmoid().view(-1).cpu().detach().numpy()) 
    
    PREDS = np.concatenate(PREDS)
    del model
    gc.collect()
    
    return PREDS

In [ ]:
def inference(model_paths, dataloader, device):
    final_preds = []
    for i, path in enumerate(model_paths):
        model = JigsawModel(CONFIG['config_name'])
        model.to(CONFIG['device'])
        model.load_state_dict(torch.load(path, map_location = device))
        
        print(f"Getting predictions for model {i+1}")
        preds = valid_fn(model, dataloader, device)
        final_preds.append(preds)
    
    final_preds = np.array(final_preds)
    final_preds = np.mean(final_preds, axis=0)
    return final_preds

In [ ]:
test_dataset = JigsawDataset(val_df, CONFIG['tokenizer'], max_length=CONFIG['max_length'], col = "more_toxic")
test_loader = DataLoader(test_dataset, batch_size=CONFIG['test_batch_size'],
                            num_workers=2, shuffle=False, pin_memory=True)
preds_albert_more = inference(MODEL_PATHS, test_loader, CONFIG['device'])

Getting predictions for model 1


  0%|          | 0/776 [00:00<?, ?it/s]

Getting predictions for model 2


  0%|          | 0/776 [00:00<?, ?it/s]

Getting predictions for model 3


  0%|          | 0/776 [00:00<?, ?it/s]

Getting predictions for model 4


  0%|          | 0/776 [00:00<?, ?it/s]

Getting predictions for model 5


  0%|          | 0/776 [00:00<?, ?it/s]

In [ ]:
test_dataset = JigsawDataset(val_df, CONFIG['tokenizer'], max_length=CONFIG['max_length'], col = "less_toxic")
test_loader = DataLoader(test_dataset, batch_size=CONFIG['test_batch_size'],
                            num_workers=2, shuffle=False, pin_memory=True)
preds_albert_less = inference(MODEL_PATHS, test_loader, CONFIG['device'])

Getting predictions for model 1


  0%|          | 0/776 [00:00<?, ?it/s]

Getting predictions for model 2


  0%|          | 0/776 [00:00<?, ?it/s]

Getting predictions for model 3


  0%|          | 0/776 [00:00<?, ?it/s]

Getting predictions for model 4


  0%|          | 0/776 [00:00<?, ?it/s]

Getting predictions for model 5


  0%|          | 0/776 [00:00<?, ?it/s]

In [ ]:
del test_dataset, test_loader 
gc.collect()

243

In [ ]:
predictions = preds_albert_more > preds_albert_less
val_df["truth"] = 1
val_truths = val_df.truth.values
print(f"Accuracy: {accuracy_score(val_truths, predictions)}")

Accuracy: 0.8276765559496937


In [ ]:
less_preds = (preds_albert_less + more_toxic_preds) / 2.0
more_preds = (preds_albert_more + less_toxic_preds) / 2.0

In [ ]:
predictions = pred_more_toxic > pred_less_toxic

In [ ]:
val_df["truth"] = 1

In [ ]:
val_truths = val_df.truth.values
print(f"Accuracy: {accuracy_score(val_truths, predictions)}")

Accuracy: 0.7173895517574976
